In [22]:
import sqlite3
import csv
# Establish a connection to the database (create a new database if it doesn't exist)
conn = sqlite3.connect('cars.db')
# Create a cursor object to execute SQL queries
cursor = conn.cursor()
# Create the Listing table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS listings (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        model VARCHAR(30),
        make VARCHAR(30),
        year INTEGER,
        mileage INTEGER,
        transmission VARCHAR(30),
        fuel_type VARCHAR(30),
        engine VARCHAR(30),
        body_type VARCHAR(30),
        vehicle_title VARCHAR(100),
        page_title VARCHAR(100),
        price DECIMAL,
        sold_date DATETIME,
        url VARCHAR(255)
    )
''')
# Read data from the CSV file (assuming the file name is 'data.csv')
with open('data/cleaned-data.csv', 'r') as file:
    csv_data = csv.DictReader(file)
    # Iterate over each row in the CSV file
    for row in csv_data:
        # Insert the data into the Listing table
        cursor.execute('''
            INSERT INTO listings (model, make, year, mileage, transmission, fuel_type, engine, body_type, vehicle_title, page_title, price, sold_date, url)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            row['model'], row['make'], row["year"], row["mileage"], row['transmission'], row["fuel_type"], row["engine"],
            row["body_type"], row["vehicle_title"], row["page_title"], row["price"], row["sold_date"], row["url"]
        ))
conn.commit()
conn.close()
print("Database has been populated successfully.")

Database has been populated successfully.


In [23]:
# Establish a connection to the database
conn = sqlite3.connect('cars.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Query to retrieve table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")

# Fetch all the table names
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])


listings
sqlite_sequence


In [16]:
import sqlite3

# Establish a connection to the database
conn = sqlite3.connect('cars.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Query to retrieve the contents of a table
table_name = 'Listings'  # Replace with the actual table name
cursor.execute(f"SELECT * FROM {table_name}")

# Fetch all the rows from the table
rows = cursor.fetchall()

# Print the column names
column_names = [description[0] for description in cursor.description]
print(column_names)

# Print the contents of the table
for row in rows:
    print(row)

['id', 'model_id', 'year', 'mileage', 'transmission_id', 'fuel_type', 'engine', 'body_type', 'vehicle_title', 'page_title', 'price', 'sold_date', 'url']
(1, 1, 2019, 51900, 1, 'Gasoline', '3.6L Gas V6', 'Sedan', 'Rebuilt, Rebuildable & Reconstructed', '2019 Cadillac CT6 PREMIUM LUXURY', 1.3, '2023-05-25', 'https://www.ebay.com/itm/185913370660?hash=item2b494ce824:g:UGQAAOSwGn5j~pIv')
(2, 3, 2016, 78000, 1, 'Gasoline', '5.7L Gas V8', 'SUV', 'Clean', '2016 Toyota Land Cruiser ', 55400, '2023-06-05', 'https://www.ebay.com/itm/266273540664?hash=item3dff23ca38:g:bK0AAOSwWw1kcXhv')
(3, 5, 2018, 42900, 1, 'Gasoline', '3.0L Gas V6', 'Convertible', 'Salvage', '2018 Jaguar F-Type P400 AWD CONVERTIABLE MSRP $102K NO RESERVE !!', 42000, '2023-06-04', 'https://www.ebay.com/itm/204351768498?hash=item2f945087b2:g:IdIAAOSwT1Fkc7P5')
(4, 7, 2017, 29000, 1, 'Gasoline', '2.5L Gas I4', 'Sedan', 'Rebuilt, Rebuildable & Reconstructed', '2017 Nissan Altima 2.5', 5850, '2023-06-12', 'https://www.ebay.com/itm/

In [24]:
import sqlite3
# Establish a connection to the database
conn = sqlite3.connect('cars.db')
# Create a cursor object to execute SQL queries
cursor = conn.cursor()
# Get the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
# Iterate through each table and display columns and first 5 rows
for table in tables:
    table_name = table[0]
    print(f"Table '{table_name}':")
    
    # Query to get column information
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    
    # Display column names
    col_names = [column[1] for column in columns]
    print(f"  {' | '.join(col_names)}")
    
    # Query to get the first 5 rows
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
    rows = cursor.fetchall()
    
    # Display rows
    for row in rows:
        print(f"  {' | '.join(map(str, row))}")
    
    print("")  # Add an empty line between tables
# Close the connection
conn.commit()
conn.close()
print("Database connection closed")

Table 'listings':
  id | model | make | year | mileage | transmission | fuel_type | engine | body_type | vehicle_title | page_title | price | sold_date | url
  1 | CT6 | Cadillac | 2019 | 51900 | Automatic | Gasoline | 3.6L Gas V6 | Sedan | Rebuilt, Rebuildable & Reconstructed | 2019 Cadillac CT6 PREMIUM LUXURY | 1.3 | 2023-05-25 | https://www.ebay.com/itm/185913370660?hash=item2b494ce824:g:UGQAAOSwGn5j~pIv
  2 | Land Cruiser | Toyota | 2016 | 78000 | Automatic | Gasoline | 5.7L Gas V8 | SUV | Clean | 2016 Toyota Land Cruiser  | 55400 | 2023-06-05 | https://www.ebay.com/itm/266273540664?hash=item3dff23ca38:g:bK0AAOSwWw1kcXhv
  3 | F-Type | Jaguar | 2018 | 42900 | Automatic | Gasoline | 3.0L Gas V6 | Convertible | Salvage | 2018 Jaguar F-Type P400 AWD CONVERTIABLE MSRP $102K NO RESERVE !! | 42000 | 2023-06-04 | https://www.ebay.com/itm/204351768498?hash=item2f945087b2:g:IdIAAOSwT1Fkc7P5
  4 | Altima | Nissan | 2017 | 29000 | Automatic | Gasoline | 2.5L Gas I4 | Sedan | Rebuilt, Rebuilda